In [2]:
!pip install -q mediapipe


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import time
import random
import numpy

In [8]:
show_finger_dots = True  # Flag to control whether fingertip dots are shown on screen

# Import necessary MediaPipe classes for hand tracking
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
HandLandmarkerResult = mp.tasks.vision.HandLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode

# Variables to store the index finger coordinates of two hands
index_finger_coords_person_A = None
index_finger_coords_person_B = None

# Callback function to handle hand detection results
def print_result(result: HandLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
    global index_finger_coords_person_A, index_finger_coords_person_B

    # Reset coordinates on each call
    index_finger_coords_person_A = None
    index_finger_coords_person_B = None

    # Process landmarks if available
    if result.hand_landmarks and result.handedness:
        for hand_landmarks, handedness in zip(result.hand_landmarks, result.handedness):
            index_tip = hand_landmarks[8]  # Index finger tip landmark
            label = handedness[0].category_name  # "Left" or "Right"

            # Assign coordinates based on handedness
            if label == "Left":
                index_finger_coords_person_A = (index_tip.x, index_tip.y)
            elif label == "Right":
                index_finger_coords_person_B = (index_tip.x, index_tip.y)

# Configure the hand landmarker for live video input and callback on detection
options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path='hand_landmarker.task'),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=print_result,
    num_hands=2)

# Create the hand landmarker instance
landmarker = HandLandmarker.create_from_options(options)

# Start video capture from the webcam
cap = cv2.VideoCapture(0)

# Exit if webcam is not found or cannot be opened
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Read a single frame to get image dimensions
ret, frame = cap.read()
h, w, _ = frame.shape

# Randomize target objective position
x_pix_objective = int(random.uniform(0.1, 0.9) * w) 
y_pix_objective = int(random.uniform(0.1, 0.9) * h)

# Initialize scores and timers for both players
points_A = 0
points_B = 0
last_toggle_time_A = 0  # Timestamp for player A's toggle delay
last_toggle_time_B = 0  # Timestamp for player B's toggle delay
hover_start_time_A = None  # Hover start time for player A
hover_start_time_B = None  # Hover start time for player B

# Main loop to process video frames
while True:
    ret, frame = cap.read()  # Capture new frame from webcam
    frame = cv2.flip(frame, 1)  # Mirror the image

    # Convert OpenCV frame to MediaPipe Image format
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
    timestamp_ms = int(time.time() * 1000)

    # Perform asynchronous hand landmark detection
    landmarker.detect_async(mp_image, timestamp_ms)

    # Process player A's finger if detected
    if index_finger_coords_person_A:
        # Convert normalized coordinates to pixel coordinates
        x_pix_finger = int(index_finger_coords_person_A[0] * w)
        y_pix_finger = int(index_finger_coords_person_A[1] * h)

        # Draw a green dot on the fingertip if toggled on
        if show_finger_dots:
            cv2.circle(frame, (x_pix_finger, y_pix_finger), 10, (0, 255, 0), -1)

        # If fingertip touches red target, award a point and generate a new target
        if numpy.sqrt((x_pix_finger - x_pix_objective)**2 + (y_pix_finger - y_pix_objective)**2) < 10:
            points_A += 1
            x_pix_objective = int(random.uniform(0.1, 0.9) * w) 
            y_pix_objective = int(random.uniform(0.1, 0.9) * h)

        # If fingertip hovers over the toggle dot, toggle visibility of dot on fingertip with a 1-second cooldown
        if numpy.sqrt((x_pix_finger - 10)**2 + (y_pix_finger - 80)**2) < 10:
            current_time = time.time()
            if current_time - last_toggle_time_A >= 1:
                show_finger_dots = not show_finger_dots
                last_toggle_time_A = current_time

        # If fingertip hovers over the quit dot for 2+ seconds, break loop
        if numpy.sqrt((x_pix_finger - 10)**2 + (y_pix_finger - 110)**2) < 10:
            current_time = time.time()
            if hover_start_time_A is None:
                hover_start_time_A = current_time
            elif current_time - hover_start_time_A >= 2:
                break
        else:
            hover_start_time_A = None

    # Process player B's finger if detected
    if index_finger_coords_person_B:
        # Convert normalized coordinates to pixel coordinates
        x_pix_finger = int(index_finger_coords_person_B[0] * w)
        y_pix_finger = int(index_finger_coords_person_B[1] * h)

        # Draw a blue dot on the fingertip if toggled on
        if show_finger_dots:
            cv2.circle(frame, (x_pix_finger, y_pix_finger), 10, (255, 0, 0), -1)

        # If fingertip touches red target, award a point and generate a new target
        if numpy.sqrt((x_pix_finger - x_pix_objective)**2 + (y_pix_finger - y_pix_objective)**2) < 10:
            points_B += 1
            x_pix_objective = int(random.uniform(0.1, 0.9) * w) 
            y_pix_objective = int(random.uniform(0.1, 0.9) * h)

        # If fingertip hovers over the toggle dot, toggle visibility of dot on fingertip with a 1-second cooldown
        if numpy.sqrt((x_pix_finger - 10)**2 + (y_pix_finger - 80)**2) < 10:
            current_time = time.time()
            if current_time - last_toggle_time_B >= 1:
                show_finger_dots = not show_finger_dots
                last_toggle_time_B = current_time

        # If fingertip hovers over the quit dot for 2+ seconds, break loop
        if numpy.sqrt((x_pix_finger - 10)**2 + (y_pix_finger - 110)**2) < 10:
            current_time = time.time()
            if hover_start_time_B is None:
                hover_start_time_B = current_time
            elif current_time - hover_start_time_B >= 2:
                break
        else:
            hover_start_time_B = None

    #  Draw red target dot, toggle dot, and quit dot
    cv2.circle(frame, (x_pix_objective, y_pix_objective), 10, (0, 0, 255), -1)  # Target
    cv2.circle(frame, (10, 80), 10, (0, 0, 0), -1)  # Toggle display dot
    cv2.circle(frame, (10, 110), 10, (0, 0, 0), -1)  # Quit dot

    # Draw score text for player A
    score_text = f"Player 1: {points_A}"
    cv2.putText(frame, score_text, org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=1, color=(0, 0, 0), thickness=2, lineType=cv2.LINE_AA)

    #  Draw score text for player B
    score_text = f"Player 2: {points_B}"
    cv2.putText(frame, score_text, org=(10, 60), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=1, color=(0, 0, 0), thickness=2, lineType=cv2.LINE_AA)

    # Draw hitbox for toggle dot
    score_text = "Toggle dot"
    cv2.putText(frame, score_text, org=(25, 90), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=1, color=(0, 0, 0), thickness=2, lineType=cv2.LINE_AA)

    # Draw hitbox for quit dot
    score_text = "Quit"
    cv2.putText(frame, score_text, org=(25, 120), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=1, color=(0, 0, 0), thickness=2, lineType=cv2.LINE_AA)

    # Show the annotated frame
    cv2.imshow("Hand Landmarker", frame)

    # Allow early exit using 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up resources
cap.release()
cv2.destroyAllWindows()